# **Computational Creativity Project**

---

### **Generative Adversarial Networks for Image Generation**

Saiham Rahman, Student ID: 210769593


In [ ]:
#@markdown #**Install libraries** 
# @markdown This cell will take a little while because it has to download several libraries.

#@markdown ---

# !pip install --upgrade torch==1.9.1+cu111 torchvision==0.10.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install --upgrade https://download.pytorch.org/whl/nightly/cu111/torch-1.11.0.dev20211012%2Bcu111-cp37-cp37m-linux_x86_64.whl https://download.pytorch.org/whl/nightly/cu111/torchvision-0.12.0.dev20211012%2Bcu111-cp37-cp37m-linux_x86_64.whl
!git clone https://github.com/NVlabs/stylegan3
# !git clone https://github.com/openai/CLIP
# !pip install -e ./CLIP
!pip install einops ninja

import sys
# sys.path.append('./CLIP')
sys.path.append('./stylegan3')

import io
import os, time, glob
import pickle
import shutil
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
import requests
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
# import clip
import unicodedata
import re
from tqdm.notebook import tqdm
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from IPython.display import display
from einops import rearrange
from google.colab import files
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.utils import get_file
from tensorflow.keras import applications
from tensorflow.keras.utils import plot_model
from keras import Model
from tensorflow.keras.optimizers import SGD
# from IPython import display

device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

print(tf.__version__)

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

# Training for StyleGAN3


In [ ]:
# @markdown #**Uncomment to Connect google colab**
def connect_gdrive():
  try:
      from google.colab import drive
      drive.mount('/content/drive', force_remount=True)
      COLAB = True
      print("Note: using Google CoLab")
  except:
      print("Note: not using Google CoLab")
      COLAB = False
# connect_gdrive() #UNCOMMENT  

In [ ]:
# @markdown #**Loading Custom Data and Resizing Images**
# @markdown Please uncomment the lines of code marked #UNCOMMENT to make it work
from PIL import Image
import os, sys
#set file path
path = "/content/drive/MyDrive/images/" #@param
save_path = "/content/drive/MyDrive/processed/" #@param
# dirs = os.listdir( path ) #UNCOMMENT

SIZE = "1024" #@param [256, 1024]
def resize():
    
    #loop through the images
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            
            #set the output size: 256,256 or 1024,1024
            imResize = im.resize((SIZE,SIZE), Image.ANTIALIAS)
            
            #save the images as image name + resized
            imResize.save(save_path + os.path.basename(item) + '_resized.jpg', 'JPEG', quality=100, optimize=True)
#run the funtion
# resize() #UNCOMMENT
# !python /content/stylegan3/dataset_tool.py --source /content/drive/MyDrive/processed --dest /content/drive/MyDrive/metfaces #UNCOMMENT

In [ ]:
import os
#@markdown #**Training Loop for StyleGAN3**

# @markdown #**Uncomment last line in code to initiate training**
#@markdown Modify these according to https://github.com/NVlabs/stylegan3/blob/main/docs/configs.md
EXPERIMENTS = "/content/drive/MyDrive/experiments_metfaces" #@param
DATA = "/content/drive/MyDrive/metfaces_128" #@param
SNAP = 10 #@param
CONFIG = "stylegan3-r" #@param ["stylegan3-r", "stylegan3-t"]
BATCH = 32 #@param 
GAMMA = 0.5 #@param
MIRROR = 1 #@param
KIMG =  5000#@param
#@markdown *For loading papers pretrainned network on ffhq dataset*
TRANSFER_LEARNING_RESUME = "True" #@param ["True", "False"]
IMAGE_SIZE = "1024" #@param [256, 1024]
TRANSFER_RESUME_NETWORK = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhqu-"+str(SIZE)+"x"+str(SIZE)+".pkl"
#@markdown ---
#@markdown **For loading previously trainned network**
PRETRAINNED_RESUME = "False" #@param ["True", "False"]
NETWORK = "network-snapshot-000000.pkl" #@param
NETWORK_FOLDER = "00000--auto1" #@param
RESUME_NETWORK = os.path.join(EXPERIMENTS, "00000--auto1", NETWORK) 

# Build the command and run it
if TRANSFER_LEARNING_RESUME == True and PRETRAINNED_RESUME == False:
  cmd = f"/usr/bin/python3 /content/stylegan3/train.py --outdir={EXPERIMENTS} --cfg={CONFIG} --data={DATA} --gpus=1 --batch={BATCH} --batch-gpu=4 --gamma={GAMMA} --mirror={MIRROR} --kimg={KIMG} --snap={SNAP} --resume={TRANSFER_RESUME_NETWORK}"
elif PRETRAINNED_RESUME == True:
  cmd = f"/usr/bin/python3 /content/stylegan3/train.py --outdir={EXPERIMENTS} --cfg={CONFIG} --data={DATA} --gpus=1 --batch={BATCH} --batch-gpu=4 --gamma={GAMMA} --mirror={MIRROR} --kimg={KIMG} --snap={SNAP} --resume={RESUME_NETWORK}"
else:
  cmd = f"/usr/bin/python3 /content/stylegan3/train.py --outdir={EXPERIMENTS} --cfg={CONFIG} --data={DATA} --gpus=1 --batch={BATCH} --batch-gpu=16 --gamma={GAMMA} --mirror={MIRROR} --kimg={KIMG} --snap={SNAP}"


# !{cmd} # EXECUTE TRAINING

# Testing and generating images using StyleGAN3

In [ ]:
#@title Generate an image
#@markdown StyleGAN3 pre-trained models for config T (translation equiv.) and config R (translation and rotation equiv.)

OUTDIR = "/content/seeds"

# Create directory
try:
    os.makedirs(OUTDIR)
except OSError:
    pass

SEED_VALUE = 5088 #@param {type:"slider", min:1000, max:9999, step:1}

baselink ='https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/'
model = "stylegan3-r-metfaces-1024x1024.pkl" #@param ["Trained Model","stylegan3-r-metfaces-1024x1024.pkl","stylegan3-r-metfacesu-1024x1024.pkl","stylegan3-t-metfaces-1024x1024.pkl","stylegan3-t-metfacesu-1024x1024.pkl"]
#@markdown Test with own model?
OWN_MODEL = "False" #@param ["True", "False"]

if OWN_MODEL == True:
# Generate an image using own trained model
  connect_gdrive()
  NETWORK = "network-snapshot-000000.pkl" #@param
  NETWORK_FOLDER = "00000--auto1" #@param
  MODEL_PATH = os.path.join("/content/drive/MyDrive/", "00000--auto1", NETWORK) 
  cmd = f"/usr/bin/python3 /content/stylegan3/gen_images.py --outdir={OUTDIR} --trunc=1 --seeds={SEED_VALUE} --network={MODEL_PATH}"
  # !python gen_images.py --outdir=out --trunc=1 \
  # --seeds=$seed --network=$MODEL_PATH
else:
# Generate an image using pre-trained model
  cmd = f"/usr/bin/python3 /content/stylegan3/gen_images.py --outdir={OUTDIR} --trunc=1 --seeds={SEED_VALUE} --network={baselink+model}"
  # !python gen_images.py --outdir=out --trunc=1 \
  # --seeds=$seed --network=$baselink$model

!{cmd}

plt.figure(figsize=(10,10))
gan_img = Image.open(f'{OUTDIR}/seed%04d.png' % SEED_VALUE);
gan_img.save(f'{OUTDIR}/seed%04d.png' % SEED_VALUE) # Save it
plt.imshow(gan_img);
plt.axis('off');

# StyleTransfer

In [ ]:
#@markdown #**Define necessary functions for StyleTransfer**
from google.colab import drive
import tensorflow as tf
import keras
import numpy as np
from tensorflow.keras.utils import get_file
import matplotlib.pyplot as plt
from tensorflow.keras import applications
from tensorflow.keras.utils import plot_model
from keras import Model
from tensorflow.keras.optimizers import SGD
#from keras.optimizers import SGD
from IPython import display

device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

print(tf.__version__)

network_choice = "vgg16" #@param['vgg19', 'vgg16']

switcher1 = {
    "vgg19": applications.vgg19,
    "vgg16": applications.vgg16,
}
switcher2 = {
    "vgg19": applications.vgg19.VGG19(weights="imagenet", include_top=False),
    "vgg16": applications.vgg16.VGG16(weights="imagenet", include_top=False),
}
network = switcher1[network_choice]

def build_model():
    model = switcher2[network_choice]
    model.summary()
    return model


# Display images function

def display_images(content_image, style_image, combination_image):
    """
    Displays content, style and combination images, labelled. Any could be missing if set to None
    """
    _, cells = plt.subplots(1, 3, figsize=(45,15))
    if content_image is not None:
        cells[0].imshow(content_image)
        cells[0].set_title("Content", fontsize=30)
    if style_image is not None:
        cells[1].imshow(style_image)
        cells[1].set_title("Style", fontsize=30)
    if combination_image is not None:
        cells[2].imshow(combination_image)
        cells[2].set_title("Combination", fontsize=30)
    for cell in cells:
        cell.axis('off')
    plt.show()

def preprocess_image(image_path):
    """
    Open, resize and format pictures into appropriate tensors
    """
    # Load image with given size into PIL format
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_height,img_width)
    )  
    # Turn image into Numpy array
    img = keras.preprocessing.image.img_to_array(img)
    # The next step expects a batch of images, so add another dimension
    img = np.expand_dims(img, axis=0)  
    # Call the network preprocessing step. 
    # Converts to BGR and zero-centers data with regards to the Imagenet dataset, by subtracting the mean channel values in Imagenet
    img = network.preprocess_input(img)  
    # Transform to tensor
    return tf.convert_to_tensor(img)  

def deprocess_image(tensor):
    """
    Reverse the preprocessing step to turn the tensor into an RGB Numpy array which we can visualise
    """
    # Transform the tensor into a Numpy array
    img = tensor.numpy()[0]

    # Return BGR values to normal (non-zero-centered), adding back in the means of the Imagenet dataset
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68

    # Convert from BGR to RGB
    img = img[:, :, ::-1]

    # Ensure values are in valid ranges
    img = np.clip(img, 0, 255).astype("uint8")

    return img

# Loss function

def gram_matrix(input_tensor):
    """
    Calculate the Gram matrix for given input
    """
    input_tensor = tf.transpose(input_tensor, (2, 0, 1))  # Transpose
    features = tf.reshape(input_tensor, (tf.shape(input_tensor)[0], -1))  # Flatten layer
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style, combination):
    """
    The style loss is the mean square error between the Gram matrix of the style image features and the Gram matrix of the combination image features,
    divided by 4 in original paper (see link at the end of the notebook)
    """
    S = gram_matrix(style)
    C = gram_matrix(combination)
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (img_size ** 2))

def content_loss(content, combination):
    """
    The content loss is the mean square error between the combination and the content image features
    """
    return tf.reduce_sum(tf.square(combination - content))

def loss_function(combination_image, content_image, style_image):
    """
    Calculate the loss function given a content image, a style image and the combination result
    """
    # Combine all the images in the same tensor
    input_tensor = tf.concat(
        [content_image, style_image, combination_image], axis=0
    )

    # Get the features in all the layers for the three images
    features = feature_extractor(input_tensor)

    # Initialise the loss
    loss = tf.zeros(shape=())

    # Extract the content layers and calculate the content loss
    for layer_name in content_layers:
        layer_features = features[layer_name]
        content_image_features = layer_features[0, :, :, :]  # Content image is at position 0
        combination_image_features = layer_features[2, :, :, :]  # Combination image is at position 2
        loss += content_weight * content_loss(content_image_features, combination_image_features)

    # Extract the style layers and calculate the style loss
    for layer_name in style_layers:
        layer_features = features[layer_name]
        style_image_features = layer_features[1, :, :, :]  # Style image is at position 1
        combination_image_features = layer_features[2, :, :, :]
        loss += style_weight * style_loss(style_image_features, combination_image_features)

    return loss

def compute_loss_and_grads(content_image, style_image, combination_image):
    """
    Brings together the calculation of loss and that of gradients
    """
    with tf.GradientTape() as tape:
        loss = loss_function(combination_image, content_image, style_image)
        grads = tape.gradient(loss, combination_image)
    return loss, grads

def save(epoch, img_tensor, save_image=True, save_model=False):
  drive.mount('/content/gdrive')
  # Directory where the checkpoints will be saved
  path = '/content/gdrive/My Drive/Work/Colab/StyleTransfer/' #@param{type: 'string'}

  model_name = path + "model_" + str(epoch) + '.tf'
  image_name = path + "img_" + str(epoch) + '.png'

  # Save image
  if save_image:
      img = deprocess_image(img_tensor)
      keras.preprocessing.image.save_img(image_name, img)

  # Save model
  if save_model:
      tf.saved_model.save(model, model_name)


In [ ]:
#@title Load chosen images and preprocess

style_image_url = "https://i.imgur.com/osgpBSch.jpg" # @param {type: "string"}
GAN_IMAGE_FILENAME = f"{SEED_VALUE}" #@param ["f\"{SEED_VALUE}\"", "\"seed4011\""] {type:"raw", allow-input: true}
content_image_path = "/content/seeds/seed"+ GAN_IMAGE_FILENAME +".png"

style_image_path = get_file(fname = "style_image.jpg", origin = style_image_url) # TODO

content_image = keras.preprocessing.image.load_img(content_image_path)
style_image = keras.preprocessing.image.load_img(style_image_path)

# display
display_images(content_image, style_image, None)

#@title Image dimensions

# Turn images into Numpy arrays to extract full dimensions
content_image_np = keras.preprocessing.image.img_to_array(content_image)
style_image_np = keras.preprocessing.image.img_to_array(style_image)

# Setting image dimensions variables
height = content_image_np.shape[0]
width = content_image_np.shape[1]
channels = content_image_np.shape[2]
# Set height
img_height = 400  #@param{type: 'number'}
# Scale width appropriately
img_width = int(width * img_height / height) 
img_size = img_height * img_width

#@title Test preprocessing
tensor = preprocess_image(style_image_path)
img = deprocess_image(tensor)

In [ ]:
#@title Build the model
model = build_model()

# Set up a model to extract features, given input, for each layer in the network
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)


In [ ]:
#@title Define the optimizer

initial_learning_rate = 100.0 #@param
decay_steps = 100 #@param
decay_rate = 0.96 #@param

optimizer = SGD(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_learning_rate, decay_steps=decay_steps, decay_rate=decay_rate))


In [ ]:
#@title Set which layers of the network should be taken into consideration when calculating content and style loss
style_layers = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layers = ["block5_conv2"]

# Total loss is a weighted sum of content and style loss
content_weight = 2.5e-8  #@param
# style_weight = 1e-6
style_weight = 1e-6  #@param
content_weight /= len(content_layers)
style_weight /= len(style_layers)



In [ ]:
#@title Compile the Model
# Compile the model
model.compile(optimizer, loss_function)

In [ ]:
#@title Optimization loop
style_transfer_save_path = f"/content/styletransfer/seed{SEED_VALUE}/"
import os
try:
    os.makedirs(style_transfer_save_path)
except OSError:
    pass

content_image_tensor = preprocess_image(content_image_path)
style_image_tensor = preprocess_image(style_image_path)
combination_image_tensor = tf.Variable(preprocess_image(content_image_path))

iterations =   500#@param{type: 'number'}

for it in range(0, iterations + 1):
    # Step 1: calculate loss and gradients
    loss, grads = compute_loss_and_grads(content_image_tensor, style_image_tensor, combination_image_tensor)
    # Step 2: apply gradients
    optimizer.apply_gradients([(grads, combination_image_tensor)])
    
    # Display images
    # display.clear_output()  # This line clears output between iterations
    if it % 100 == 0:
      print("Iteration %d: loss=%.2f" % (it, loss))
      # display_images(content_image, style_image, deprocess_image(combination_image_tensor))
      # display_images(None,deprocess_image(combination_image_tensor),None)
      keras.preprocessing.image.save_img(style_transfer_save_path + f"seed{SEED_VALUE}_iter%d.png"%(it), deprocess_image(combination_image_tensor))
    if it == iterations:
      display_images(content_image, style_image, deprocess_image(combination_image_tensor))

    # Save image and/or model
    #save(it, combination_image_tensor, save_image=False, save_model=False)

In [ ]:
#@title Using Tensorflow HUB
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
# Using the numpy arrays representing our images, with 1 batch dimension added, and normalized
c_img = np.expand_dims(content_image_np, axis=0)/255 
s_img = np.expand_dims(style_image_np, axis=0)/255 

stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0])
keras.preprocessing.image.save_img(style_transfer_save_path + f"seed{SEED_VALUE}_TF_HUB.png", stylized_image[0])

# MISCALANIOUS Codes

In [ ]:
#@title code for making collage
# from PIL import Image
# import numpy as np
# # open images by providing path of images
# img1 = Image.open("/content/fake_tot/seed2363_TF_HUB.png") 
# img2 = Image.open("/content/fake_tot/seed3676_TF_HUB.png")
# img3 = Image.open("/content/fake_tot/seed4089_TF_HUB.png")
# img4 = Image.open("/content/fake_tot/seed5088_TF_HUB.png")
# img5 = Image.open("/content/fake_tot/seed6463_TF_HUB.png")
# img6 = Image.open("/content/fake_tot/seed8001_TF_HUB.png")
# #create arrays of above images
# img1_array = np.array(img1)
# img2_array = np.array(img2)
# img3_array = np.array(img3)
# img4_array = np.array(img4)
# img5_array = np.array(img5)
# img6_array = np.array(img6)
# # ====== collage of 4 images ====== 
# # arrange arrays of four images in two rows
# imgg1 = np.vstack([np.hstack([img1_array , img2_array, img3_array]) , np.hstack([img4_array , img5_array, img6_array])]) 
# #create image of imgg1 array
# finalimg1 = Image.fromarray(imgg1)
# #provide the path with name for finalimg1 where you want to save it
# finalimg1.save("/content/collage.png")
# print("Second image saved")